In [8]:
# !pip install langchain==0.1.13 langchainhub==0.1.15 python-dotenv==1.0.1 huggingface_hub==0.22.2
# !pip install sentence-transformers
# !pip install groq
# !pip install langchain_groq
# !pip install google-search-results
#!pip install faiss-cpu

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.5 MB 10.5 MB/s eta 0:00:02
   ----- ---------------------------------- 2.1/14.5 MB 22.8 MB/s eta 0:00:01
   ---------- ----------------------------- 3.8/14.5 MB 26.9 MB/s eta 0:00:01
   --------------- ------------------------ 5.5/14.5 MB 29.4 MB/s eta 0:00:01
   ------------------- -------------------- 7.2/14.5 MB 30.5 MB/s eta 0:00:01
   ------------------------ --------------- 8.9/14.5 MB 28.5 MB/s eta 0:00:01
   ---------------------------- ----------- 10.4/14.5 MB 32.8 MB/s eta 0:00:01
   --------------------------------- ------ 12.1/14.5 MB 34.4 MB/s eta 0:00:01
   ------------------------------------- -- 13.7/14.5 MB 34.4 MB/s eta 0:00:01
   ---------------------------------------  14.5/14.5 MB 32.8 MB/s eta 0:00:01
   ---------------------------------------  14.5/14.5 MB 32.8 MB/s eta 0:00:01
   ---------------------------------------- 14.5/14.5 MB 24.2 MB/s 

In [2]:
from langchain.agents import load_tools
from groq import Groq
import string
import os

client = Groq(
    api_key = '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'
)

from langchain_groq import ChatGroq
from langchain.agents import initialize_agent
llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192")



In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system","You are a world class technical documentation writer"
        ),
        (
            "user","{input}"
        )
    ]
)


chain = prompt | llm


chain.invoke("how cna langsmith help with tetsing")

AIMessage(content="Langsmith is an innovative AI-powered tool that can significantly aid in testing by generating test cases, test data, and even entire test scripts. Here are some ways Langsmith can help with testing:\n\n1. **Automated Test Case Generation**: Langsmith can analyze your code and generate test cases based on the functionality, identifying potential edge cases and scenarios that might be difficult to cover manually.\n\n2. **Test Data Generation**: Langsmith can generate test data, including input values, expected outputs, and sample data, reducing the time and effort required to create test data manually.\n\n3. **Test Script Generation**: Langsmith can generate entire test scripts in popular testing frameworks like JUnit, PyUnit, or TestNG, saving developers a significant amount of time and effort.\n\n4. **Code Review and Analysis**: Langsmith can analyze your codebase, identifying potential issues, inconsistencies, and areas for improvement, ensuring that your code is r

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser
chain.invoke({"input":"how can langsmith help with testing"})

"Langsmith! As a technical documentation writer, I'm excited to explore how Langsmith can facilitate testing. Langsmith is an AI-powered tool that helps automate the process of creating and maintaining technical documentation. Here are some ways Langsmith can assist with testing:\n\n1. **Automated documentation testing**: Langsmith can automatically generate documentation for APIs, SDKs, and other technical systems. This automated documentation can be used as a source of truth for testing, ensuring that the documentation accurately reflects the system's behavior.\n\n2. **Test case generation**: Langsmith's AI capabilities can analyze the documentation and generate test cases based on the documented functionality. This can save time and effort in creating test cases, ensuring that the testing process is more efficient.\n\n3. **Documentation-driven testing**: Langsmith's documentation can serve as a single source of truth for testing. By using the documentation as a reference, testers ca

In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import sentence_transformers
embeddings = HuggingFaceEmbeddings()

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

d:\DEV\WebdevFolder\RealEstateAI\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\DEV\WebdevFolder\RealEstateAI\.venv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PC-User\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator.

In [7]:
vector = FAISS.from_documents(documents,embeddings)


from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

{context}


Question: {input}""")

document_chain = create_stuff_documents_chain(llm,prompt)


from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()

retrieval_chain = create_retrieval_chain(retriever, document_chain)


response = retrieval_chain.invoke({"input":"how can langsmith help with testing"})
print(response["answer"])

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system","You are a world class technical documentation writer"
        ),
        (
            "user","{input}"
        )
    ]
)


from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),
    ("user","Given the above conversation, generate a search query to look up in order to get information")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)


from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can Langsmith help test my LLm applications?"),AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history":chat_history,
    "input":"Tell me how"
})

[Document(page_content="Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace‚ÄãWe provide multiple ways to log traces to LangSmith. Below, we'll highlight", metadata={'source': 'https://docs.smith.lang

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system","Answer the user's questions based on the below context\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),

])
document_chain = create_stuff_documents_chain(llm,prompt)
retrieval_chain = create_retrieval_chain(retriever_chain,document_chain)


from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can Langsmith help test my LLm applications?"),AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history":chat_history,
    "input":"Tell me how"
})

{'chat_history': [HumanMessage(content='Can Langsmith help test my LLm applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-a

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for info about LangSmith. For any questions about LangSmith, use this tool!"
)


from langchain_community.tools.tavily_search import TavilySearchResults


os.environ['TAVILY_API_KEY'] = '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'


search = TavilySearchResults(tavily_api_key = os.getenv('TAVILY_API_KEY'))


tools = [search]

In [ ]:
from langchain_groq import ChatGroq
from langchain import hub
from langchain.agents import create_structured_chat_agent
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, ChatMessagePromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, PromptTemplate
import typing
import langchain_core
#prompt = hub.pull("hwchase17/structured-chat-agent")
prompt = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    input_types={
        'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]
        },
    metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'structured-chat-agent', 'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'},
    messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'],
    template='Respond to the human as helpfully and accurately as possible. BE AS SPECIFIC AND DETAILED AS POSSIBLE. Your answer must be long and you must perform a giant web search, that is, not superficial. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per \
JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Your answer must be long and you need to perform enough searches with the searching tool to get enough information. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation. Please put all your observations into the final answer IT MUST BE DETAILED FINAL ANSWER')),
    MessagesPlaceholder(variable_name='chat_history', optional=True),
    HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'],
    template='{input}\n\n{agent_scratchpad}\n (reminder to respond in a JSON blob no matter what)'))]
)
agent = create_structured_chat_agent(llm,tools,prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
system_msg = """
                You are an expert at questioning the client about their renovation or contstruction project quote.
                Please keep in mind all of these things when asking questions: ask the user specific info needed for the quote, such as:

                    1.Property Details:
                    Address and location of the property.
                    Type of property (e.g., residential, commercial).
                    Age and current condition of the property.
                    Square footage of the area to be renovated.
                    2.Scope of the Project:
                    Detailed description of the renovation work required.
                    Specific areas or rooms that need renovation.
                    Desired changes and additions (e.g., new rooms, extensions, wall removal).
                    Level of finishes desired (standard, premium, luxury).
                    3.Design Preferences:
                    Style or theme of the renovation (modern, traditional, industrial, etc.).
                    Color schemes and material preferences.
                    Any specific fixtures or features desired (e.g., type of flooring, lighting fixtures).
                    4.Budget:
                    Your budget range for the project.
                    Any flexibility within the budget.
                    5.Timeline:
                    Desired start date and completion date.
                    Any constraints or deadlines (e.g., events planned at the property).
                    Legal and Compliance Information:
                    Information on any permits or approvals already obtained.
                    Any restrictions or caveats affecting the property.
                    6.Utility Modifications:
                    Changes or improvements to electrical, plumbing, or HVAC systems.
                    Details of any provisions for appliances or systems (e.g., energy-efficient or smart home features).
                    7.Access & Logistics:
                    Access requirements for renovation or construction work (e.g., elevator access, restricted hours).
                    Logistic considerations necessary for the project (e.g., space for storing materials, parking for construction vehicles).
                    8.Special Requirements:
                    Access provisions for renovations or construction work done on the property.
                    Special considerations or instructions (e.g., conserving existing architectural features).
                    What specific materials they are looking at.
                    Information about other construction or renovation projects completed on the property.
                    IMPORTANT!: Make sure you get clear answers from the user that can actually be used for making the quote, and if you are not provided a clear answer ask to clarify, and restate the question and explain what it means.
                    IMPORTANT!: Ask each question that you have ONE BY ONE

                    IMPORTANT!: WHEN YOU HAVE ALL THE INFORMATION, JUST SAY 'hehehe' at the end.

                """


chat_history = []
while True:


    client_req = str(input())
    quotebot = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": system_msg

            },

            {
                "role": "user",
                "content": client_req + "Chat History for your own context and info - DONT ASK A QUESTION IF THE ANSWER IS ALREADY IN CHAT HISTORY: " + str(chat_history)
            }
        ],

        model="llama3-70b-8192",
    )
    output = (quotebot.choices[0].message.content)
    print(output)
    chat_history.append(output)
    if('hehehe' in output.lower()): break

k
Let's get started. I'll ask the questions one by one to ensure I get all the necessary information for the renovation or construction project quote.

Here's my first question:

What is the address and location of the property you'd like to renovate or construct?
13582 cobra drive Herndon VA
It seems like we've already discussed the address and location of the property, which is 13582 Cobra Drive, Herndon, VA. Let's move on to the next question!

What type of property is this? Is it a residential or commercial property?
residential
Since we already have the address and location, let's move on to the next question.

You mentioned it's located at 13582 Cobra Drive, Herndon, VA. Now, you've already told me the property type, which is residential. 

Let's proceed to the next question.

Can you please tell me the age and current condition of the property? For example, is it an old house that needs a lot of work, or is it relatively new and well-maintained?
It was built in 1990 and is in fa

In [ ]:
consultors_list = []
consultationbot = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": system_msg

            },

            {
                "role": "user",
                "content": "You are a proffesional renovation consultor. Based on the chat history, create a streamlined material plan for the bathroom, choosing what materials to use, and how much would be used and where. MAKE SURE TO BE VERY SPECIFIC in what materials you will use and how much. ALSO MAKE SURE TO PROVIDE A BLURB AT THE START OF THE RESPOnse And THEN A MATERIAL LIST AND HOW MUCH YOU WILL NEEED. DO NOT LIST PRICES JUST LIST MATERIALS NEEDED AND HOW MUCH OF THAT MATERIAL" + "Here is the chat history:" + str(chat_history)

            }
        ],
        model="llama3-70b-8192",
    )
consoltation_output = (consultationbot.choices[0].message.content)
consultors_list.append(consoltation_output)



quotebot = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": system_msg

            },

            {
                "role": "user",
                "content": "Based on the chat history as well as the consultors list of materials, you must put all the required information for the quote into a streamlined format so that a web search query can be formed for it." + "Chat History for your own context and info: " + str(chat_history) + "Consultors List: " + str(consultors_list)
            }
        ],
        model="llama3-70b-8192",
    )
streamlined_output = (quotebot.choices[0].message.content)






In [ ]:
streamlined_output

'Here is the streamlined format for the quote:\n\n**Property Details:**\n\n* Address: 13582 Cobra Drive, Herndon, VA\n* Type: Residential\n* Age and Condition: Not specified\n* Square Footage: 150 sq ft (basement area to be renovated)\n\n**Scope of the Project:**\n\n* Renovate 150 sq ft area in the basement\n* Add nice wooden flooring, bright overhead lighting, and shelves lining one side of the wall\n* Install a 100-inch TV, a beautiful mirror, a large couch, a game area with a pool table and foosball table, and a hot tub behind a tinted glass wall\n\n**Design Preferences:**\n\n* Modern style\n* Not specified (additional details needed)\n\n**Budget:**\n\n* $500,000\n* Flexibility within the budget: Not specified (additional details needed)\n\n**Timeline:**\n\n* Completion within 2 months\n* Specific events or deadlines driving the timeline: Not specified (additional details needed)\n\n**Legal and Compliance Information:**\n\n* Permits or approvals obtained: Not specified (additional d

In [ ]:
consoltation_output

"**Material Plan for the Bathroom Renovation**\n\nBased on our conversation, I understand that you're looking to renovate a 150 sq ft area in your basement with modern features like a 100-inch TV, a beautiful mirror, a large couch, a game area with a pool table and foosball table, and a hot tub behind a tinted glass wall. Here's a material plan tailored to your needs:\n\n**Flooring:**\n\n* 150 sq ft of engineered hardwood flooring (e.g., oak or maple) for the main area\n* 50 sq ft of luxury vinyl tile (LVT) for the hot tub area\n\n**Walls:**\n\n* 150 sq ft of drywall for the main area\n* 50 sq ft of glass blocks for the hot tub area\n\n**Lighting:**\n\n* 12 LED recessed lights for the main area\n* 4 LED strip lights for under-cabinet lighting\n* 1 large chandelier for the game area\n\n**Painting and Finishing:**\n\n* 2 gallons of premium paint for the main area (modern color scheme)\n* 1 gallon of clear coat for the wooden flooring\n* 1 can of textured spray paint for the ceiling\n\n**

In [ ]:
output = agent_executor.invoke({"input":"Given the chat history --> "+streamlined_output+"<-- AS WELL AS THE CONSULTANT'S INFORMATION -->" + consoltation_output + " --> look for labor and material costs for whatever the user asked for in the location of the user's property. maybe also use the costs of houses or properties very near to that location to decide on the cost. BE VERY SPECIFIC. LOTS OF NUMBERS. Also for material costs only use the consoltation_output, and search up the materials individually to find the price."})



> Entering new AgentExecutor chain...
Action:```
{
  "action": "tavily_search_results_json",
  "action_input": "Renovation quote for a 150 sq ft basement area in a residential property at 13582 Cobra Drive, Herndon, VA, with modern design, premium finishes, and specific requirements for flooring, walls, lighting, cabinets, plumbing, and electrical systems, within a budget of $500,000 and a timeline of 2 months."
}
```[{'url': 'https://sebringdesignbuild.com/modern-basement-ideas-to-prompt-your-own-remodel/', 'content': "RELAX. This ultra-contemporary basement area is ideal for a lady's secret hideaway. Have a glass of wine with the ladies down below in the basement! The use of gold, white, and brown in this basement design creates a luxurious space. What a fantastic job Dwelling Designs did on this basement retreat for ladies."}, {'url': 'https://www.architecturaldigest.com/reviews/home-improvement/cost-to-finish-basement', 'content': 'However, when you buy something through our reta

In [ ]:
output

{'input': 'Given the chat history --> Here is the streamlined format for the quote:\n\n**Property Details:**\n\n* Address: 13582 Cobra Drive, Herndon, VA\n* Type: Residential\n* Age and Condition: Not specified\n* Square Footage: 150 sq ft (basement area to be renovated)\n\n**Scope of the Project:**\n\n* Renovate 150 sq ft area in the basement\n* Add nice wooden flooring, bright overhead lighting, and shelves lining one side of the wall\n* Install a 100-inch TV, a beautiful mirror, a large couch, a game area with a pool table and foosball table, and a hot tub behind a tinted glass wall\n\n**Design Preferences:**\n\n* Modern style\n* Not specified (additional details needed)\n\n**Budget:**\n\n* $500,000\n* Flexibility within the budget: Not specified (additional details needed)\n\n**Timeline:**\n\n* Completion within 2 months\n* Specific events or deadlines driving the timeline: Not specified (additional details needed)\n\n**Legal and Compliance Information:**\n\n* Permits or approvals 

In [ ]:
output["output"]

{'response': 'Based on the provided information, I have calculated the estimated renovation cost for the 150 sq ft basement area in the residential property at 13582 Cobra Drive, Herndon, VA.\n\n**Material Costs:**\n\n* 150 sq ft of engineered hardwood flooring (e.g., oak or maple): $3,000 - $5,000\n* 50 sq ft of luxury vinyl tile (LVT) for the hot tub area: $1,500 - $3,000\n* 150 sq ft of drywall for the main area: $500 - $1,000\n* 50 sq ft of glass blocks for the hot tub area: $1,000 - $2,000\n* 12 LED recessed lights for the main area: $1,200 - $2,400\n* 4 LED strip lights for under-cabinet lighting: $200 - $400\n* 1 large chandelier for the game area: $500 - $1,000\n* 2 gallons of premium paint for the main area: $100 - $200\n* 1 gallon of clear coat for the wooden flooring: $50 - $100\n* 1 can of textured spray paint for the ceiling: $20 - $50\n* 10 linear ft of premium cabinetry for storage and shelving: $2,000 - $4,000\n* 2 large shelves for the game area: $500 - $1,000\n* 2 new

In [ ]:
newstr = ""
ctr = 0
for word in output["output"]["response"].split(" "):
    if(ctr % 6 == 0 and ctr != 0):
        newstr += "\n"
    newstr += word + " "
    ctr += 1

print(newstr)

Based on the provided information, I 
have calculated the estimated renovation cost 
for the 150 sq ft basement 
area in the residential property at 
13582 Cobra Drive, Herndon, VA.

**Material Costs:**

* 
150 sq ft of engineered hardwood 
flooring (e.g., oak or maple): $3,000 
- $5,000
* 50 sq ft of 
luxury vinyl tile (LVT) for the 
hot tub area: $1,500 - $3,000
* 
150 sq ft of drywall for 
the main area: $500 - $1,000
* 
50 sq ft of glass blocks 
for the hot tub area: $1,000 
- $2,000
* 12 LED recessed lights 
for the main area: $1,200 - 
$2,400
* 4 LED strip lights for 
under-cabinet lighting: $200 - $400
* 1 
large chandelier for the game area: 
$500 - $1,000
* 2 gallons of 
premium paint for the main area: 
$100 - $200
* 1 gallon of 
clear coat for the wooden flooring: 
$50 - $100
* 1 can of 
textured spray paint for the ceiling: 
$20 - $50
* 10 linear ft 
of premium cabinetry for storage and 
shelving: $2,000 - $4,000
* 2 large 
shelves for the game area: $500 
- $1,000
* 2 new 

In [ ]:
quotebot2 = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": "You are an expert in PREPARING A REAL ESTATE QUOTE IN PROPER FORMAT from ONLY what is in the user's request, given a web search synthesis as one input and user's request as another input"

            },

            {
                "role": "user",
                "content": "Based on the input, which is <<<"+newstr+">>> you must parse the input for the renovation quote so that your response talks about ONLY the stuff relevant to the user's quote request. IMPORTANT: GIVE AN EXACT AMOUNT FOR MATERIAL VALUE & LABOR COST " + "USer's request for your own context and info: " + str(output)
            }
        ],
        model="llama3-70b-8192",
    )
output2 = (quotebot2.choices[0].message.content)
print(output2)

Based on the user's request, I have prepared a detailed estimate for the renovation project in the specified format:

**Property Details:**
* Address: 13582 Cobra Drive, Herndon, VA
* Type: Residential
* Age and Condition: Not specified
* Square Footage: 150 sq ft (basement area to be renovated)

**Labor Costs:**
The total labor cost for the project is estimated to be around $175,000, considering the complexity of the project and the timeline of 2 months.

**Material Costs:**
The material costs for the project are estimated as follows:

* Flooring: $12,500 (engineered hardwood flooring and luxury vinyl tile)
* Walls: $7,500 (drywall and glass blocks)
* Lighting: $4,000 (LED recessed lights, LED strip lights, and chandelier)
* Painting and Finishing: $3,000 (premium paint and clear coat)
* Cabinets and Shelves: $10,000 (premium cabinetry and shelving)
* Plumbing and Electrical: $12,500 (new vents, outlets, switches, and hot tub plumbing kit)
* Miscellaneous: $12,500 (mirror, TV mount, p

In [ ]:
from langchain import LLMMathChain

calc = LLMMathChain.from_llm(llm,verbose=True)
while False:
  try:
    calc.run("""Add up all the money Flooring: $12,500 (engineered hardwood flooring and luxury vinyl tile)
* Walls: $7,500 (drywall and glass blocks)
* Lighting: $4,000 (LED recessed lights, LED strip lights, and chandelier)
* Painting and Finishing: $3,000 (premium paint and clear coat)
* Cabinets and Shelves: $10,000 (premium cabinetry and shelving)
* Plumbing and Electrical: $12,500 (new vents, outlets, switches, and hot tub plumbing kit)
* Miscellaneous: $12,500 (mirror, TV mount, pool table, foosball table, and hot tub)""")
  except:
    continue

Streaming output truncated to the last 5000 lines.
...numexpr.evaluate("12500 + 7500 + 4000 + 3000 + 10000 + 12500 + 12500")...


> Entering new LLMMathChain chain...
Add up all the money Flooring: $12,500 (engineered hardwood flooring and luxury vinyl tile)
* Walls: $7,500 (drywall and glass blocks)
* Lighting: $4,000 (LED recessed lights, LED strip lights, and chandelier)
* Painting and Finishing: $3,000 (premium paint and clear coat)
* Cabinets and Shelves: $10,000 (premium cabinetry and shelving) 
* Plumbing and Electrical: $12,500 (new vents, outlets, switches, and hot tub plumbing kit)
* Miscellaneous: $12,500 (mirror, TV mount, pool table, foosball table, and hot tub)Here is the math problem translated into a Python expression using the numexpr library:

```text
12500 + 7500 + 4000 + 3000 + 10000 + 12500 + 12500
```
...numexpr.evaluate("12500 + 7500 + 4000 + 3000 + 10000 + 12500 + 12500")...


> Entering new LLMMathChain chain...
Add up all the money Flooring: $12,500 (engineere